REMEMBER: FIRST CREATE A COPY OF THIS FILE WITH A UNIQUE NAME AND DO YOUR WORK THERE. AND MAKE SURE YOU COMMIT YOUR CHANGES TO THE `hw3_submissions` BRANCH.

# Assignment 3 | Cleaning and Exploring Data with Pandas



<img src="data/scoreCard.jpg" width=250>

In this assignment, you will investigate restaurant food safety scores for restaurants in San Francisco. Above is a sample score card for a restaurant. The scores and violation information have been made available by the San Francisco Department of Public Health. 

## Loading Food Safety Data


There are 2 files in the data directory:
1. business.csv containing food establishments in San Francisco
1. inspections.csv containing retaurant inspections records

Let's start by loading them into Pandas dataframes.  One of the files, business.csv, has encoding (ISO-8859-1), so you will need to account for that when reading it.

### Question 1

#### Question 1a
Read the two files noted above into two pandas dataframes named `bus` and `ins`, respectively. Print the first 5 rows of each to inspect them.


In [2]:
import pandas as pd

In [3]:
bus = pd.read_csv('data/businesses.csv', encoding='ISO-8859-1')
ins = pd.read_csv('data/inspections.csv')

In [4]:
bus.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


In [5]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


## Examining the Business data

From its name alone, we expect the `businesses.csv` file to contain information about the restaurants. Let's investigate this dataset.

### Question 2

#### Question 2a: How many records are there?

In [6]:
len(bus)

6315

#### Question 2b: How many unique business IDs are there?  

In [7]:
bus['business_id'].nunique()

6315

#### Question 2c: What are the 5 most common businesses by name, and how many are there in San Francisco?

In [8]:
bus['name'].value_counts().nlargest(5)

name
STARBUCKS COFFEE              72
PEET'S COFFEE & TEA           24
MCDONALDS                     12
SAN FRANCISCO SOUP COMPANY    11
WALGREENS                     11
Name: count, dtype: int64

## Zip code

Next, let's  explore some of the variables in the business table. We begin by examining the postal code.

### Question 3

#### Question 3a
How are the zip code values stored in python (i.e. data type)?

To answer this you might want to examine a particular entry.

In [9]:
type(bus.at[4,'postal_code'])

str

#### Question 3b

What are the unique values of postal_code?

In [13]:
bus['postal_code'].unique()

array(['94104', '94109', '94133', '94110', '94122', '94103', '94115',
       '94131', '94111', '94117', '94107', '94108', '94102', '94132',
       '94105', '94134', '94124', '94116', '94121', '94118', '94112',
       '94123', '94114', '94127', '941102019', '941', '94545', '94066',
       '941033148', nan, '94158', '95105', '94013', '94130', 'CA',
       '92672', '94120', '94143', '94609', '94101', '00000', '94188',
       '94621', '94014', '94129', '94602', 'Ca'], dtype=object)

#### Question 3c

Let's say we decide to exclude the businesses that have no zipcode for our analysis (which might include food trucks for example).  Use the list of valid 5-digit zip codes below to create a new dataframe called bus_valid, with only businesses whose postal_codes show up in this list of valid zipcodes. How many businesses are there in this new dataframe?

In [14]:
validZip = ["94102", "94103", "94104", "94105", "94107", "94108",
            "94109", "94110", "94111", "94112", "94114", "94115",
            "94116", "94117", "94118", "94121", "94122", "94123", 
            "94124", "94127", "94131", "94132", "94133", "94134"]

In [15]:
bus_valid = bus[bus['postal_code'].isin(validZip)]
bus_valid.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494
3,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN
4,45,CHARLIE'S DELI CAFE,3202 FOLSOM ST,San Francisco,CA,94110,37.747114,-122.413641,+14156415051


## Latitude and Longitude

Another aspect of the data we want to consider is the prevalence of missing values. If many records have missing values then we might be concerned about whether the nonmissing values are representative of the population.

### Question 4
 
Consider the longitude and latitude in the business DataFrame. 

#### Question 4a

How many businesses are missing longitude values, working with only the businesses that are in the list of valid zipcodes?

In [16]:
null_lon_df = bus_valid[pd.isnull(bus_valid['longitude'])]
len(null_lon_df)

2483

In [18]:
null_lon_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number
200,821,LA SANTANECA,3781 MISSION ST,San Francisco,CA,94110,NaN,NaN,+14156481034
210,852,NEW HOLLYWOOD BAKERY & RESTAURANT,652 PACIFIC AVE,San Francisco,CA,94133,NaN,NaN,+14153979919
760,2934,LOU'S FISH SHACK,300 JEFFERSON ST,San Francisco,CA,94133,NaN,NaN,+14157715687
826,3123,LES JOULINS,44 ELLIS ST,San Francisco,CA,94102,NaN,NaN,+14153975397
1100,4323,DIVA'S,1081 POST ST,San Francisco,CA,94109,NaN,NaN,+14154743482


#### Question 4b

Create a new dataframe with one row for each valid zipcode. The dataframe should include the following three columns:

1. `postal_code`: Contains the zip codes in the `validZip` variable above.
2. `null_lon`: The number of businesses in that zipcode with missing `longitude` values.
3. `not_null_lon`: The number of businesses without missing `longitude` values.

In [20]:
null_lon_count_df = null_lon_df['longitude'].isnull().groupby(null_lon_df['postal_code']).sum().reset_index(name='total_businesses')
not_null_lon_df = bus_valid['longitude'].notnull().groupby(bus_valid['postal_code']).sum().reset_index(name='missing_lon_count')

zip_lon_df = pd.merge(null_lon_count_df, not_null_lon_df, on='postal_code', how='inner')
zip_lon_df.head()

,postal_code,total_businesses,missing_lon_count
0,94102,196,262
1,94103,252,306
2,94104,59,74
3,94105,100,126
4,94107,247,204


#### 4c. Do any zip codes appear to have more than their 'fair share' of missing longitude? 

To answer this, you will want to compute the proportion of missing longitude values for each zip code, and print the proportion missing longitude, and print the top five zipcodes in descending order of proportion missing postal_code.


In [27]:
total_businesses = null_lon_df.groupby('postal_code').size().reset_index(name='total_businesses')

missing_lon_count = null_lon_df['longitude'].isnull().groupby(null_lon_df['postal_code']).sum().reset_index(name='missing_lon_count')

zip_lon_df = pd.merge(total_businesses, missing_lon_count, on='postal_code', how='left')

zip_lon_df['proportion_missing'] = zip_lon_df['missing_lon_count'] / zip_lon_df['total_businesses']

top_missing_zipcodes = zip_lon_df.sort_values(by='proportion_missing', ascending=False).head()

print(top_missing_zipcodes[['postal_code', 'proportion_missing']])

   postal_code  proportion_missing
0        94102                 1.0
1        94103                 1.0
22       94133                 1.0
21       94132                 1.0
20       94131                 1.0


# Investigate the inspection data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns, these are named `business_id`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

### Question 5

#### Question 5a
As with the business data, assess whether there is one inspection record for each business, by counting how many rows are in the data and how many unique businesses there are in the data. If they are exactly the same number, it means there is only one inspection per business, clearly.

In [28]:
ins.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


In [29]:
len(ins)

15430

#### Question 5b

What values does `type` take on? How many occurrences of each value is in the DataFrame? Create a new dataframe named `ins2` by copying `ins` and keeping only records with values of `type` that occur more than 10 times in the original table. In other words, eliminate records that have values of `type` that occur rarely (< 10 times). Check the result to make sure rare types are eliminated.

In [30]:
ins.dtypes

business_id     int64
score           int64
date            int64
type           object
dtype: object

In [31]:
ins['type'].value_counts()

type
routine      15429
complaint        1
Name: count, dtype: int64

In [34]:
ins2 = ins.loc[ins['type'] == 'routine']
ins2.head()

,business_id,score,date,type
0,10,82,20160503,routine
1,10,94,20140729,routine
2,10,92,20140114,routine
3,19,94,20160513,routine
4,19,94,20141110,routine


#### Question 5c

Since the data was stored in a .csv file, the dates are formatted as strings such as `20160503`. Once we read in the data, we would like to have dates in an appropriate format for analysis. Add a new column called `year` by capturing the first four characters of the date column. 

Hint: we have seen multiple ways of doing this in class, includings `str` operations, `lambda` functions, `datetime` operations, and others. Choose the method that works best for you :)

In [38]:
ins2.loc['year'] = ins2['date'].str.split().str[-1].str[:4]
ins2 = ins2.drop('date', errors = 'ignore')
ins2.head()

,business_id,score,date,type,year
0,10.0,82.0,20160503,routine,2016
1,10.0,94.0,20140729,routine,2014
2,10.0,92.0,20140114,routine,2014
3,19.0,94.0,20160513,routine,2016
4,19.0,94.0,20141110,routine,2014


#### Question 5d

What range of years is covered in this data set? Are there roughly same number of inspections each year? Try dropping records for any years with less than 50 inspections and store the result in a new dataframe named `ins3`.

In [42]:
ins2['year'] = pd.to_numeric(ins2['year'], errors='coerce').astype('Int64')
min_year = ins2['year'].min()
max_year = ins2['year'].max()
print(min_year)
print(max_year)

2013
2016


In [43]:
year_count = ins2['year'].value_counts()
print(year_count)

year
2016    5839
2014    5629
2015    3923
2013      38
Name: count, dtype: Int64


In [45]:
ins3 = ins2[ins2['year'].isin(year_count[year_count>50].index)]
ins3.head()

,business_id,score,date,type,year
0,10.0,82.0,20160503,routine,2016
1,10.0,94.0,20140729,routine,2014
2,10.0,92.0,20140114,routine,2014
3,19.0,94.0,20160513,routine,2016
4,19.0,94.0,20141110,routine,2014


Let's examine only the inspections for one year: 2016. This puts businesses on a more equal footing because [inspection guidelines](https://www.sfdph.org/dph/eh/Food/Inspections.asp) generally refer to how many inspections should occur in a given year.

In [47]:
ins3_2016 = ins3[ins3['year'] == 2016]
ins3_2016.head()

,business_id,score,date,type,year
0,10.0,82.0,20160503,routine,2016
3,19.0,94.0,20160513,routine,2016
6,24.0,98.0,20161005,routine,2016
7,24.0,96.0,20160311,routine,2016
10,31.0,84.0,20160816,routine,2016


### Question 6

#### Question 6a

Merge the business and 2016 inspections data, keeping all businesses regardless of whether they show up in the inspections file. Show the first several rows of the resulting dataframe.

In [49]:
bus_ins_merge = pd.merge (bus, ins3_2016, on= 'business_id', how = 'left')
bus_ins_merge.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,phone_number,score,date,type,year
0,10,TIRAMISU KITCHEN,033 BELDEN PL,San Francisco,CA,94104,37.791116,-122.403816,+14154217044,82.0,20160503,routine,2016
1,19,NRGIZE LIFESTYLE CAFE,"1200 VAN NESS AVE, 3RD FLOOR",San Francisco,CA,94109,37.786848,-122.421547,+14157763262,94.0,20160513,routine,2016
2,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,98.0,20161005,routine,2016
3,24,OMNI S.F. HOTEL - 2ND FLOOR PANTRY,"500 CALIFORNIA ST, 2ND FLOOR",San Francisco,CA,94104,37.792888,-122.403135,+14156779494,96.0,20160311,routine,2016
4,31,NORMAN'S ICE CREAM AND FREEZES,2801 LEAVENWORTH ST,San Francisco,CA,94133,37.807155,-122.419004,NaN,84.0,20160816,routine,2016


#### Question 6b
Print the 20 lowest rated businesses names, their addresses, and their ratings.

In [50]:
bus_ins_merge[['name', 'address', 'score']].nsmallest(20, 'score')

,name,address,score
4892,GOLDEN RIVER RESTAURANT,5827 GEARY BLVD,52.0
4873,UNCLE CAFE,65 WAVERLY PL,55.0
6281,CRAZY PEPPER,2257 SAN JOSE AVE,55.0
7324,POKI TIME,2101 LOMBARD ST,55.0
3008,HING WANG BAKERY,339 JUDAH ST,56.0
5627,GOLDEN WOK,295 B ORIZABA AVE,56.0
549,MA'S DIM SUM & CAFE,1315 POWELL ST,57.0
836,RED A BAKERY,634 CLEMENT ST,57.0
4209,SAN TUNG RESTAURANT LLC,1031 IRVING ST,57.0
4903,"NEW GARDEN RESTAURANT, INC.",716 KEARNY ST,57.0


## Done!

Now commit this notebook to your `hw3_submissions` branch, push it to your GitHub repo, and open a PR!

:)